<a href="https://colab.research.google.com/github/w4bo/2022-bbs-dsaa/blob/master/01-DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data preprocessing

Pandas is the shorthand for 'Python and Data Analysis'. It provides a rich set of features for exploring and manipulating data, making it the go-to toolkit for a lot of data scientists.

Check also: https://pandas.pydata.org/

In [ ]:
import numpy as np
import pandas as pd
print(pd.__version__)

Creating some pandas series...

In [ ]:
ser_a = pd.Series([1, 2, 3, 4], index=["a", "b", "c", "d"])
ser_b = pd.Series([1, 2, 3, 4], index=["b", "a", "c", "d"])

ser_a + ser_b

... doing some element-wise operations

In [ ]:
ser_a + ser_b
ser_a - ser_b
ser_a * ser_b
ser_a / ser_b

... doing some aggregation 

In [ ]:
ser_c = pd.Series([1, np.nan, 3, 4], index=["a", "b", "c", "d"])
print(ser_c.dtype)               # Which dtype has `ser_c`?

ser_c.count()             # => 3
ser_c.sum()               # => 8
ser_c.mean()              # => 2.67
ser_c.mean(skipna=False)  # => nan
ser_c.max()               # => 4
ser_c.min()               # => 1
ser_c.idxmax()            # => "d"

ser_d = pd.Series([1, "a", 3, 4], index=["a", "b", "c", "d"])
print(ser_d.dtype)        # which dtype has `ser_d`?

ser_e = pd.Series([1, 1, 1, np.nan, 3, 4])
ser_e

Cheking unique values

In [ ]:
ser_e.unique() # => [ 1., nan,  3.,  4.]
ser_e.nunique() # => 3
ser_e.value_counts()

In [ ]:
ser_e.hist(bins=10)

Checking null values

In [ ]:
df = pd.DataFrame() # create an empty dataframe
df["ser_e"] = ser_e # add the column "ser_e"
df["isna"] = ser_e.isna() # add the column "isna", True if the value is NaN
df["notna"] = ser_e.notna() # add the column "notna", False if the value is NaN
df

In [ ]:
ser_e.mean() # the mean is 2
# fill the missing value with the average
df["mean"] = ser_e.fillna(ser_e.mean())
df

In [ ]:
# fill the missing value with the previous (not NaN) value
df["ffill"] = ser_e.fillna(method="ffill")
df

In [ ]:
# fill the missing value with the following (not NaN) value
df["bfill"] = ser_e.fillna(method="bfill")
df

In [ ]:
ser_e.dropna()

Creating pandas dataframes

In [ ]:
# create a dataframe with random data
import random
random.seed(3)
df = pd.DataFrame([[random.randint(0, 9) for i in range(10)] for i in range(5)],
                  index=[i for i in range(5)], 
                  columns=list('abcdefghij'))
df

Doing some operations with pandas dataframes

In [ ]:
# Select a column (i.e., a series)
df['a']

# Add another column
df['k'] = df['a'] * df['b']

# Get the headers (i.e., the column names)
df.columns

# Get just the first two rows
df.head(2)

# Get just the last two rows
df.tail(2)

# Sort the dataframe by columns
df.sort_values(by=['a', 'b'], ascending=[False, True])

# To get some statistics (e.g., count, mean, std, min, etc.)
df.describe()

Why is the `mean` around 4.0 and 5.0?

#### Real dataset

Importing the data from a csv file.
Check also: https://www.kaggle.com/camnugent/california-housing-prices

The data contains information from the 1990 California census. It does provide an accessible introductory dataset for teaching people about the basics of machine learning.

From https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/
- This data has metrics such as the population, median income, median housing price, and so on for each block group in California.
- Block groups are the smallest geographical unit for which the US Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people). We will just call them “districts” for short
- The goal is to build a model to predict the median housing price in any district, given all the other metrics

In [ ]:
# df = pd.read_csv("datasets/2022-bbs-dsaa-housing.csv", delimiter=",")
df = pd.read_csv("http://big.csr.unibo.it/projects/nosql-datasets/2022-bbs-dsaa-housing.csv", delimiter=",")
df

Dataset description

1. `longitude`: A measure of how far west a house is; a higher value is farther west
2. `latitude`: A measure of how far north a house is; a higher value is farther north
3. `housingMedianAge`: Median age of a house within a block; a lower number is a newer building
4. `totalRooms`: Total number of rooms within a block
5. `totalBedrooms`: Total number of bedrooms within a block
6. `population`: Total number of people residing within a block
7. `households`: Total number of households, a group of people residing within a home unit, for a block
8. `medianIncome`: Median income for households within a block of houses (measured in tens of thousands of US Dollars)
9. `medianHouseValue`: Median house value for households within a block (measured in US Dollars)
10. `oceanProximity`: Location of the house w.r.t ocean/sea

In [ ]:
df.info() # show some statistics on the dataframe

#### Memory usage

What if I change float64 to float32?

In [ ]:
dff = df.copy(deep=True) # copy the dataframe
for x in df.columns: # iterate over the columns
    if dff[x].dtype == 'float64': # if the column has type `float64`
        dff[x] = dff[x].astype('float32') # ... change it to `float32`
dff.info() # show some statistics on the dataframe

In [ ]:
df.describe()

#### Missing values
There are some missing values for `total_bedrooms`. What should we do?

Most Machine Learning algorithms cannot work with missing features. We have three options:
- Get rid of the corresponding districts (i.e., drop the rows)
    - `df.dropna(subset=["total_bedrooms"])`
- Get rid of the whole attribute (i.e., drop the columns)
    - `df.drop("total_bedrooms", axis=1`
- Set the values to some value (zero, the mean, the median, etc.)
    - `df["total_bedrooms"].fillna(df["total_bedrooms"].median())`

#### Non-numeric attributes
`ocean_proximity` is a text attribute so we cannot compute its median. Some options:
- Get rid of the whole attribute. (`df.drop("ocean_proximity", axis=1`)
- Change from categorical to ordinal (e.g., `NEAR BAY` = 0, `INLAND` = 1)
    - Can foresee any problem in this?
    - ML algorithms will assume that two nearby values are more similar than two distant values. This may be fine in some cases (e.g., for ordered categories such as “bad”, “average”, “good”, “excellent”), but it is obviously not the case for the ocean_proximity column (for example, categories 0 and 4 are clearly more similar than categories 0 and 1). 
- Change from categorical to one hot encoding
    - To fix this issue, a common solution is to create one binary attribute per category: one attribute equal to 1 when the category is “<1H OCEAN” (and 0 otherwise), another attribute equal to 1 when the category is “INLAND” (and 0 otherwise), and so on. This is called one-hot encoding, because only one attribute will be equal to 1 (hot), while the others will be 0 (cold). The new attributes are sometimes called dummy attributes

In [ ]:
df["ocean_proximity"].value_counts()

Change from categorical to ordinal

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
y = ordinal_encoder.fit_transform(df[["ocean_proximity"]])
y

From categorical to one-hot encoding

In [ ]:
y = pd.get_dummies(df["ocean_proximity"], prefix='ocean_proximity')
y

### Visualization

Visualizing attribute distributions

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
df.hist(bins=50, figsize=(20,15))
plt.show()

Open questions:

- `median_income` should be in dollars. However, it has a strange range. Why? "you are told that the data has been scaled and capped at 15 (actually 15.0001) for higher median incomes, and at 0.5 (actually 0.4999) for lower median incomes. The numbers represent roughly tens of thousands of dollars. The numbers represent roughly tens of thousands of dollars"
- `housing_median_age` and `median_house_value` are capped. As to `median_house_value`, this is a serious problem since it is your target attribute (your labels). Your Machine Learning algorithms may learn that prices never go beyond that limit. You need to check with your client team (the team that will use your system’s output) to see if this is a problem or not. If they tell you that they need precise predictions even beyond 500,000USD, then you have mainly two options: (a) collect proper labels for the districts whose labels were capped, (b) remove those districts from the training set."
- These attributes have very different scales. Should we scale them?
- Many histograms are tail heavy: they extend much farther to the right of the median than to the left. This may make it a bit harder for some Machine Learning algorithms to detect patterns

#### Scaling attributes

Attributes have very different scales. Should we scale them?


- Normalization is good to use when you know that the distribution of your data does not follow a Gaussian distribution. This can be useful in algorithms that do not assume any distribution of the data like K-Nearest Neighbors and Neural Networks.
- Standardization, on the other hand, can be helpful in cases where the data follows a Gaussian distribution. However, this does not have to be necessarily true. Unlike normalization, standardization does not have a bounding range. So, even if you have outliers in your data, they will not be affected by standardization.


#### Min-max normalization

In [ ]:
num_df = df.drop(columns=['ocean_proximity', 'median_house_value'])
normalized_df = (num_df - num_df.min()) / (num_df.max() - num_df.min())
normalized_df

#### Standardization

In [ ]:
num_df = df.drop(columns=['ocean_proximity', 'median_house_value'])
normalized_df = (num_df - num_df.mean()) / num_df.std()
normalized_df

#### Hands on!

In [ ]:
num_df = df.copy(deep=True).drop(columns=["ocean_proximity"]) # do not change this line

# Filling in (i.e., impute) missing values with the median value 
num_df["total_bedrooms"] = 1 # change `1` with the proper solution 

# Add a new column: population_per_household = population / households
num_df["population_per_household"] = 1 # change `1` with the proper solution 

# Add a new column: rooms_per_household = total_rooms / households
num_df["rooms_per_household"] = 1 # change `1` with the proper solution 

# Add a new column: bedrooms_per_room = total_bedrooms / total_rooms
num_df["bedrooms_per_room"] = 1 # change `1` with the proper solution 

# Apply standardization to all the numeric columns
num_df = pd.DataFrame() # change `pd.DataFrame()` with the proper solution 

# One hot encode `ocean_proximity` since it is a categorical attribute 
cat_df = pd.DataFrame() # change `pd.DataFrame()` with the proper solution (hint: pd.get_dummies)

clean_df = pd.concat([num_df, cat_df], axis=1) # do not change this line
clean_df